# Shallow Profiler: Technical


This notebook provides the technical back-story for the `shallowprofiler` notebook. Here is a broad outline; 
with the idea of keeping the main data-driven narrative less cluttered.


- There are three sites that support shallow profilers
- Each site has a platform tethered at 200 meters depth
- Each platform has a winch that (nine times daily) allows the profiler to rise to near the surface and descend again
- As a result we have sensor profiles from the instruments affixed to the profiler
- We have pressure / depth data as well so we can produce profile charts
    - These have depth as the vertical access and sensor reading on the horizontal axis
    

To further reduce clutter there are Python modules available as well: `shallowprofiler.py` and so forth.


From the imported `shallowprofiler.py` module we have available `sensors`: A list of lists. 
Each entry is `[sensor_name, instrument_name]` so for example `sensors[4]` is `['temperature', 'ctd']`.
To see the entirety: `sensors`.


The data volume across many sensors get fairly large. There are multiple ways to approach
not bogging down the Jupyter/IPython environment:

- Work at full time resolution in a narrow time window (e.g. one month)
- Work at a reduced time resolution

With instrument and sensor names sorted (with spectral irradiance a special case) the next step
is to build a data dictionary called **`d `** with keys corresponding to sensor (data) types:
**`conductivity`, `temperature`, `pco2`** etcetera. The corresponding value for each sensor
key is a five-tuple indexed \[0\], \[1\], ..., \[4\].


```
0: XArray DataArray: sensor data
1: XArray DataArray: sensor depth (meters, negative down) corresponding to data
2: float: Default charting lower limit for data
3: float: Default charting upper limit for data
4: string: Default chart color e.g. "blue"
```

## Developing the data dictionary: sensor profiles


There are two types of data we are concerned with here: Sensor data and profile metadata. 
The sensor data from the profiler are viewed in relation to depth, hence 'profile charts'. 
In order to generate these we need to know when the profiler is starting and ending 
an ascent; as well as a descent and a rest period when it is parked on the platform
at 200 meters.


Profile metadata is read using the function `ReadProfileMetadata(filename)` where the filename
describes the site and the time period of interest. The data are held in a pandas Dataframe.


The data structure `sensors` is a list of 2-element lists. The first element is a sensor name
and the second is the corresponding instrument that carries the sensor. 


`sensors[3] = ['salinity', 'ctd']`


The data structure `ranges` is a dictionary of expected upper and lower bounds for each sensor.
The keys are sensor names as found in `sensors`; values are tuples.


`ranges['salinity'] = (31, 35)`


The data structure `standard_deviations` is a dictionary of standard deviation ranges, also tuples.


`standard_deviations['salinity'] = (.0, .4)`


The data structure `colors` is a dictionary of colors associated with sensors in charts.


`colors['salinity'] = 'cyan'`


The data structure `sensor_names` is a dictionary of expanded sensor names more suited
to chart labels.


`sensor_names['salinity'] = 'Salinity'`

In [ ]:
from shallowprofiler import *
from charts import *

# profiles is a pandas DataFrame treated as a global resource
# The directory tree for the Jupyter Book:
# 
# book/chapters/rob/<notebooks.ipynb>                    (where this notebook resides)
#              /rca/profiles/osb/january2022.csv
#                           /oos
#                           /axb
#
# The resulting DataFrame has columns r0t, r0z, r1t, r1z, a0t, a0z, a1t, a1z, d0t, d0z, d1t, d1z
#   corresponding to rest / ascent / descent <start - end> <depth> <time>
#
profiles = ReadProfileMetadata()


# Let's look at some typical ascent and descent timestamps
print('First slow ascent start time is local midnight:', profiles['a0t'][3], 'UTC')
print()
print("9 ascent intervals:")
for i in range(9):
    print(profiles['a1t'][i] - profiles['a0t'][i], '         depths:', profiles['a0z'][i], 'to', profiles['a1z'][i], 'meters')
print()
print("9 descent intervals:")
for i in range(9):
    print(profiles['d1t'][i] - profiles['d0t'][i], '         depths:', profiles['d0z'][i], 'to', profiles['d1z'][i], 'meters')
print()
print("9 rest intervals:")
for i in range(9):
    print(profiles['r1t'][i] - profiles['r0t'][i], '         depths:', profiles['r0z'][i], 'to', profiles['r1z'][i], 'meters')
print()

From this we can infer:

- We are inspecting the nine profile runs from 01-JAN-2022 at the Oregon Slope Base site 


- There are a couple of profiles (2 and 7 in the list of 9) that did not run full scope
    - Ascents stopped below 100 meters so ignore these
    

- Profiles 4 and 9 correspond to midnight and noon (slower) profiles 
    - Times are UTC: 7 (PST) or 8 (PDT) hours ahead of local
    

- Ascent time is about 70 minutes (for both normal and slow profiles)
    - Typical ascent depth range is 174.4 meters
    - Ascent speed is therefore 4.15 cm per second
    - Using an 8-second sample average window would give a vertical sampling of 0.3322 meters / downsample
    - This is about 525 samples per profile
    

- Descent time (normal profiles) is about 39 minutes
    - Slow profile descent time is about 100 minutes


- Rest periods are of less interest
    - They do give an opportunity to track profiler instruments against platform instruments
        - For example: Is a change in salinity reflected in both sensors?
    - Rest times prior to normal profiles seem to vary in duration (under one hour)
    - Rest times prior to midnight/noon profiles seem longer (more than one hour)
    - Rest depths vary by a couple meters start to end as one would expect from tides etcetera